In [1]:
## Standard

import os
import sys
sys.path.append('../RD-systems-and-test-benches')

import math
from datetime import datetime

## Specific

from tqdm.notebook import tqdm
import copy
# from concurrent.futures import ProcessPoolExecutor
# import networkx as nx
# from itertools import product
# from itertools import chain

## Data handling 

import pandas as pd
import numpy as np
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows

import sklearn.metrics
import scipy.constants as scc
import seaborn as sns
import scipy.integrate as integrate
import scipy.optimize as sco

import pickle

## Plot

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scienceplots
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])

import plotly.graph_objects as go
import plotly.io as pio

from IPython.core.display import HTML

## Own modumes

import model as ty
import heat_transfer as bht
import proc as pr

import plot_functions_here as pfun
import general as gen

import thermal.SK_standard as nm
import utils.plot_functions as RD_pfun
import repo_config as rconfig

import utils.data_processing as dp

## Physics

import fluids as fds
import ht 

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter
import scienceplots
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
plt.style.use(['science'])

def to_percent(y, _):
    return f"{y * 100:.0f}\%"

In [3]:
folder_path = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel'
geometry_path = os.path.join(folder_path, 'Panels\\Panel_V4.411(NI).xlsx')
hypotheses_path = folder_path + '\\' + 'Hypotheses\\Model_hypotheses_calibrated.xlsx'
# condi_path = folder_path + '\\' + 'Steady-state conditions inputs\\ssCforStudy.xlsx'
# exp_path = folder_path + '\\' + 'Experiments\\Experiments-GMI.xlsx'

panelSpecs = pr.import_geometry(geometry_path)

# panelSpecs['part2']['lambd_abs_ins'] = 0.
# panelSpecs['part2']['lambd_tube_ins'] = 0.

hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

# steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre en compte
# stepConditions = steadyStateConditions_df.to_dict(orient='records')

recap = pd.read_excel(r'G:\Drive partagés\Cercle Hard\R&D\Thermal performance tests PVT\Recap_RD_TUV_SSI.xlsx')
recap = recap.drop(index=0)
recap.dropna(how='all', inplace=True)

groups = {key: group for key, group in recap.groupby('Design')}

In [ ]:
steadyStateConditions_df = groups["V4.411(NI)"].loc[groups["V4.411(NI)"]['Type'] == "Dark front"].copy()
pr.adjust_steadyStateConditions(steadyStateConditions_df, hyp, convert_celsius=True)

steadyStateConditions_df = steadyStateConditions_df.loc[(steadyStateConditions_df['u'] != " ")]
steadyStateConditions_df['u'] = steadyStateConditions_df['u'].astype(float)
steadyStateConditions_df = steadyStateConditions_df.loc[(steadyStateConditions_df['u'] >= 0.1)]

steadyStateConditions_df.reset_index(drop=True,inplace=True)
steadyStateConditions_df

In [5]:
dic_names = {'T_in' : 'T_fluid_in',
             'T_out' : 'T_fluid_out',
             'Qdot_1_AG' : 'Qdot/AG',
             'T_amb' : 'T_amb',
             'u' : 'u'}

In [7]:
df_res, list_res = ty.simu_steadyStateConditions(panelSpecs,hyp,steadyStateConditions_df)

df_res['Qdot_exp'] = steadyStateConditions_df['Qdot']

In [ ]:
df_res[['u', 'Qdot_tube_fluid', 'Qdot_exp']]

In [9]:
df_res['relative error'] = (df_res['Qdot_exp'] - df_res['Qdot_tube_fluid']) / df_res['Qdot_exp']

In [ ]:
mae = df_res['relative error'].abs().mean()
mae

In [ ]:
sns.histplot(df_res['relative error'], bins=20)